In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder
from langchain.schema import BaseMessage, AIMessage, HumanMessage

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 600,
    chunk_overlap = 100,
)
loader = UnstructuredFileLoader("./files/chapter_two.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

memory = ConversationBufferMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True,
    )

def load_memory(_):
    chat_history = memory.load_memory_variables({})["chat_history"]

    if not chat_history:
        return []
    
    return [
        msg if isinstance(msg, BaseMessage) else HumanMessage(content=msg)
        for msg in chat_history
    ]

chain = {"context":retriver,"chat_history": RunnableLambda(lambda _: load_memory(_)),"question" : RunnablePassthrough()} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question},{"output": result.content})
    print(result)

invoke_chain("Is Aaronson guilty?")

content="I don't know."


In [11]:
invoke_chain("What message did he write in the table?")

content='He wrote: "GOD IS POWER"'


In [12]:
invoke_chain("Who is Julia?")

content='Julia is the love interest of the main character in the text.'
